In [1]:
import finetune
from torchvision import datasets, transforms
import os
import torch

In [3]:
def get_dataset():
    dataset = {}
    input_size = finetune.get_classifier_input_size()
    tforms = transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
    ])
    # tforms = finetune.get_data_transforms()['train']
    for x in ['train', 'val', 'test']:
        dataset[x] = datasets.ImageFolder(os.path.join(finetune.dataset_path(), x), transform=tforms)
    return dataset

def get_dataloaders():
    ds = get_dataset()
    dataloaders = {}
    for x in ['train', 'val', 'test']:
        dataloaders[x] = torch.utils.data.DataLoader(ds[x], batch_size=finetune.BATCH_SIZE, shuffle=True, num_workers=4)
    return dataloaders

def get_means_stddevs():
    dataloaders = get_dataloaders()
    loader = dataloaders['train']
    sum_totals = torch.zeros(3)
    variation_totals = torch.zeros(3)
    n_samples = 0
    for (inputs, labels) in loader:
        sum_totals += torch.sum(torch.mean(inputs, (-1,-2)), -2) # mean accross width, height, sum across batch
        n_samples += len(inputs)
    channel_means = sum_totals/n_samples
    print(f'means: {channel_means.detach()}')
    for (inputs, labels) in loader:
        variations = inputs - channel_means[None, :, None, None]
        variations_squared = variations * variations
        variation_totals += torch.sum(torch.mean(variations_squared, (-1,-2)), -2)
    channel_stddevs = torch.sqrt(variation_totals / n_samples)
    print(f'stddevs: {channel_stddevs}')
    return channel_means, channel_stddevs

for dataset in finetune.DATASET_OPTIONS:
    finetune.DATASET = dataset
    print(dataset)
    get_means_stddevs()

StanfordCars
means: tensor([0.4593, 0.4466, 0.4453])
stddevs: tensor([0.2919, 0.2910, 0.2988])
FGVC-Aircraft
means: tensor([0.4842, 0.5142, 0.5364])
stddevs: tensor([0.2251, 0.2185, 0.2495])
